# Football League Simulation with Tactical AI

This notebook simulates a football league with player tracking data and trains a tactical AI using BiLSTM and Multi-Head Attention to predict goal probability.

## 1. Import Required Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from dataclasses import dataclass
from typing import List, Tuple, Dict
import random
from collections import defaultdict

# Deep Learning imports
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, Model
from tensorflow.keras.layers import LSTM, Bidirectional, Dense, Dropout, Input, MultiHeadAttention, LayerNormalization
from tensorflow.keras.optimizers import Adam

# Set random seeds for reproducibility
np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

# Plotting style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 8)

## 2. Player Class

In [ ]:
class Player:
    """Represents a football player with random stats."""
    
    def __init__(self, name: str, position: str):
        self.name = name
        self.position = position  # GK, DEF, MID, FWD
        
        # Generate random stats based on position
        if position == 'GK':
            self.speed = np.random.randint(40, 70)
            self.shooting = np.random.randint(20, 40)
            self.passing = np.random.randint(40, 70)
            self.defense = np.random.randint(70, 95)
            self.dribbling = np.random.randint(20, 40)
        elif position == 'DEF':
            self.speed = np.random.randint(50, 80)
            self.shooting = np.random.randint(30, 60)
            self.passing = np.random.randint(50, 80)
            self.defense = np.random.randint(70, 95)
            self.dribbling = np.random.randint(40, 70)
        elif position == 'MID':
            self.speed = np.random.randint(60, 85)
            self.shooting = np.random.randint(50, 80)
            self.passing = np.random.randint(70, 95)
            self.defense = np.random.randint(40, 70)
            self.dribbling = np.random.randint(60, 90)
        else:  # FWD
            self.speed = np.random.randint(70, 95)
            self.shooting = np.random.randint(70, 95)
            self.passing = np.random.randint(50, 80)
            self.defense = np.random.randint(30, 50)
            self.dribbling = np.random.randint(70, 95)
        
        self.overall = int(np.mean([self.speed, self.shooting, self.passing, self.defense, self.dribbling]))
    
    def __repr__(self):
        return f"{self.name} ({self.position}, Overall: {self.overall})"
    
    def get_stats(self) -> Dict:
        return {
            'name': self.name,
            'position': self.position,
            'speed': self.speed,
            'shooting': self.shooting,
            'passing': self.passing,
            'defense': self.defense,
            'dribbling': self.dribbling,
            'overall': self.overall
        }

## 3. Team Class

In [ ]:
class Team:
    """Represents a football team with multiple players."""
    
    def __init__(self, name: str, num_players: int = 11):
        self.name = name
        self.players = []
        self.wins = 0
        self.draws = 0
        self.losses = 0
        self.goals_for = 0
        self.goals_against = 0
        
        # Create players with formation: 1 GK, 4 DEF, 4 MID, 2 FWD
        positions = ['GK'] + ['DEF'] * 4 + ['MID'] * 4 + ['FWD'] * 2
        
        for i, pos in enumerate(positions):
            player_name = f"{name}_Player_{i+1}"
            self.players.append(Player(player_name, pos))
        
        self.overall_rating = int(np.mean([p.overall for p in self.players]))
    
    def __repr__(self):
        return f"{self.name} (Rating: {self.overall_rating}, Record: {self.wins}W-{self.draws}D-{self.losses}L)"
    
    def get_points(self) -> int:
        return self.wins * 3 + self.draws
    
    def get_goal_difference(self) -> int:
        return self.goals_for - self.goals_against
    
    def record_result(self, goals_for: int, goals_against: int):
        self.goals_for += goals_for
        self.goals_against += goals_against
        
        if goals_for > goals_against:
            self.wins += 1
        elif goals_for == goals_against:
            self.draws += 1
        else:
            self.losses += 1
    
    def get_stats_df(self) -> pd.DataFrame:
        return pd.DataFrame([p.get_stats() for p in self.players])

## 4. League Class

In [ ]:
class League:
    """Represents a football league managing multiple teams."""
    
    def __init__(self, name: str, team_names: List[str]):
        self.name = name
        self.teams = [Team(team_name) for team_name in team_names]
        self.match_results = []
        self.tracking_data = []
    
    def simulate_match(self, team1: Team, team2: Team, generate_tracking: bool = True) -> Tuple[int, int, List]:
        """Simulate a match between two teams and optionally generate tracking data."""
        
        # Calculate match outcome based on team ratings with randomness
        rating_diff = team1.overall_rating - team2.overall_rating
        team1_advantage = 1 + (rating_diff / 200)  # Normalize advantage
        
        # Generate goals using Poisson distribution with team strength
        team1_goals = np.random.poisson(2 * team1_advantage)
        team2_goals = np.random.poisson(2 / team1_advantage)
        
        # Cap maximum goals at 7
        team1_goals = min(team1_goals, 7)
        team2_goals = min(team2_goals, 7)
        
        tracking = []
        if generate_tracking:
            # Generate tracking data for the match
            total_goals = team1_goals + team2_goals
            num_sequences = max(50, total_goals * 10)  # More sequences for exciting matches
            
            for seq_idx in range(num_sequences):
                sequence = self._generate_tracking_sequence(team1, team2, seq_idx < team1_goals * 10)
                tracking.append(sequence)
        
        return team1_goals, team2_goals, tracking
    
    def _generate_tracking_sequence(self, team1: Team, team2: Team, is_goal_sequence: bool) -> Dict:
        """Generate a sequence of player movements and events."""
        
        sequence_length = np.random.randint(10, 30)  # 10-30 timesteps per sequence
        num_players = 11  # Track 11 players from attacking team
        
        # Initialize positions (football pitch: 105m x 68m)
        positions = np.zeros((sequence_length, num_players, 2))
        
        # Starting positions spread across the field
        start_x = np.random.uniform(20, 50, num_players)
        start_y = np.random.uniform(10, 58, num_players)
        
        for t in range(sequence_length):
            if t == 0:
                positions[t, :, 0] = start_x
                positions[t, :, 1] = start_y
            else:
                # Players move towards goal with some randomness
                dx = np.random.uniform(-2, 3, num_players)  # Bias towards goal
                dy = np.random.uniform(-2, 2, num_players)
                
                positions[t, :, 0] = np.clip(positions[t-1, :, 0] + dx, 0, 105)
                positions[t, :, 1] = np.clip(positions[t-1, :, 1] + dy, 0, 68)
        
        # Calculate features for prediction
        # Distance to goal (center at x=105, y=34)
        distances_to_goal = np.sqrt((positions[:, :, 0] - 105)**2 + (positions[:, :, 1] - 34)**2)
        
        # Average distance (team compactness)
        centroid = positions.mean(axis=1, keepdims=True)
        spread = np.sqrt(((positions - centroid)**2).sum(axis=2)).mean(axis=1)
        
        # Events during sequence
        events = []
        for t in range(sequence_length):
            event_type = np.random.choice(['pass', 'dribble', 'shot', 'tackle'], p=[0.5, 0.3, 0.1, 0.1])
            events.append(event_type)
        
        return {
            'positions': positions,  # (timesteps, players, 2)
            'distances_to_goal': distances_to_goal,  # (timesteps, players)
            'spread': spread,  # (timesteps,)
            'events': events,
            'is_goal': 1 if is_goal_sequence else 0,
            'team1_rating': team1.overall_rating,
            'team2_rating': team2.overall_rating
        }
    
    def simulate_season(self):
        """Simulate a full season where each team plays every other team twice."""
        print(f"Simulating {self.name} season...\n")
        
        match_count = 0
        for i, team1 in enumerate(self.teams):
            for j, team2 in enumerate(self.teams):
                if i != j:
                    # Simulate match
                    goals1, goals2, tracking = self.simulate_match(team1, team2, generate_tracking=True)
                    
                    # Record results
                    team1.record_result(goals1, goals2)
                    team2.record_result(goals2, goals1)
                    
                    # Store match result
                    self.match_results.append({
                        'team1': team1.name,
                        'team2': team2.name,
                        'goals1': goals1,
                        'goals2': goals2
                    })
                    
                    # Store tracking data
                    self.tracking_data.extend(tracking)
                    
                    match_count += 1
                    if match_count % 5 == 0:
                        print(f"{team1.name} {goals1} - {goals2} {team2.name}")
        
        print(f"\nSeason complete! {match_count} matches played.")
        print(f"Generated {len(self.tracking_data)} tracking sequences.")
    
    def get_standings(self) -> pd.DataFrame:
        """Get current league standings."""
        standings = []
        for team in self.teams:
            standings.append({
                'Team': team.name,
                'Played': team.wins + team.draws + team.losses,
                'Wins': team.wins,
                'Draws': team.draws,
                'Losses': team.losses,
                'GF': team.goals_for,
                'GA': team.goals_against,
                'GD': team.get_goal_difference(),
                'Points': team.get_points(),
                'Rating': team.overall_rating
            })
        
        df = pd.DataFrame(standings)
        df = df.sort_values(['Points', 'GD', 'GF'], ascending=[False, False, False])
        df['Position'] = range(1, len(df) + 1)
        
        return df[['Position', 'Team', 'Played', 'Wins', 'Draws', 'Losses', 'GF', 'GA', 'GD', 'Points', 'Rating']]
    
    def get_match_results_df(self) -> pd.DataFrame:
        """Get all match results as a DataFrame."""
        return pd.DataFrame(self.match_results)

## 5. Create League and Simulate Season

In [ ]:
# Create a league with 6 teams
team_names = ['Arsenal', 'Manchester United', 'Chelsea', 'Liverpool', 'Manchester City', 'Tottenham']
premier_league = League('Premier League', team_names)

# Display team information
print("Teams in the league:")
for team in premier_league.teams:
    print(f"  {team}")

In [ ]:
# Simulate the season
premier_league.simulate_season()

In [ ]:
# Display final standings
standings = premier_league.get_standings()
print("\n=== FINAL LEAGUE STANDINGS ===")
print(standings.to_string(index=False))

## 6. Prepare Training Data for Tactical AI

In [ ]:
def prepare_training_data(tracking_data: List[Dict], max_sequence_length: int = 30):
    """Prepare tracking data for training the tactical AI model."""
    
    X_positions = []
    X_features = []
    y_labels = []
    
    for sequence in tracking_data:
        positions = sequence['positions']  # (timesteps, players, 2)
        distances = sequence['distances_to_goal']  # (timesteps, players)
        spread = sequence['spread']  # (timesteps,)
        is_goal = sequence['is_goal']
        
        seq_len = positions.shape[0]
        
        # Pad or truncate to max_sequence_length
        if seq_len < max_sequence_length:
            # Pad
            pad_len = max_sequence_length - seq_len
            positions_padded = np.pad(positions, ((0, pad_len), (0, 0), (0, 0)), mode='constant')
            distances_padded = np.pad(distances, ((0, pad_len), (0, 0)), mode='constant')
            spread_padded = np.pad(spread, ((0, pad_len),), mode='constant')
        else:
            # Truncate
            positions_padded = positions[:max_sequence_length]
            distances_padded = distances[:max_sequence_length]
            spread_padded = spread[:max_sequence_length]
        
        # Flatten positions for each timestep: (timesteps, players * 2)
        positions_flat = positions_padded.reshape(max_sequence_length, -1)
        
        # Additional features per timestep
        features = np.concatenate([
            distances_padded,  # (timesteps, players)
            spread_padded.reshape(-1, 1),  # (timesteps, 1)
            np.full((max_sequence_length, 1), sequence['team1_rating']),
            np.full((max_sequence_length, 1), sequence['team2_rating'])
        ], axis=1)
        
        X_positions.append(positions_flat)
        X_features.append(features)
        y_labels.append(is_goal)
    
    return np.array(X_positions), np.array(X_features), np.array(y_labels)

# Prepare data
print(f"Preparing training data from {len(premier_league.tracking_data)} sequences...")
X_pos, X_feat, y = prepare_training_data(premier_league.tracking_data)

print(f"\nData shapes:")
print(f"  X_positions: {X_pos.shape}  (samples, timesteps, position_features)")
print(f"  X_features: {X_feat.shape}  (samples, timesteps, additional_features)")
print(f"  y: {y.shape}  (samples,)")
print(f"\nGoal sequences: {y.sum()} ({y.mean()*100:.1f}%)")

## 7. Build Tactical AI Model (BiLSTM + Multi-Head Attention)

In [ ]:
def build_tactical_ai_model(sequence_length: int, pos_features: int, add_features: int):
    """Build BiLSTM + Multi-Head Attention model for goal prediction."""
    
    # Input layers
    input_positions = Input(shape=(sequence_length, pos_features), name='positions')
    input_features = Input(shape=(sequence_length, add_features), name='features')
    
    # Combine inputs
    combined = layers.Concatenate()([input_positions, input_features])
    
    # BiLSTM layers to capture temporal movement patterns
    lstm1 = Bidirectional(LSTM(128, return_sequences=True))(combined)
    lstm1 = LayerNormalization()(lstm1)
    lstm1 = Dropout(0.3)(lstm1)
    
    lstm2 = Bidirectional(LSTM(64, return_sequences=True))(lstm1)
    lstm2 = LayerNormalization()(lstm2)
    lstm2 = Dropout(0.3)(lstm2)
    
    # Multi-Head Attention to capture player interactions
    attention = MultiHeadAttention(
        num_heads=4,
        key_dim=32,
        dropout=0.2
    )(lstm2, lstm2)
    attention = LayerNormalization()(attention)
    
    # Residual connection
    x = layers.Add()([lstm2, attention])
    x = Dropout(0.3)(x)
    
    # Global pooling to aggregate sequence
    x_max = layers.GlobalMaxPooling1D()(x)
    x_avg = layers.GlobalAveragePooling1D()(x)
    x = layers.Concatenate()([x_max, x_avg])
    
    # Dense layers for classification
    x = Dense(64, activation='relu')(x)
    x = Dropout(0.4)(x)
    x = Dense(32, activation='relu')(x)
    x = Dropout(0.4)(x)
    
    # Output layer - goal probability
    output = Dense(1, activation='sigmoid', name='goal_probability')(x)
    
    # Create model
    model = Model(inputs=[input_positions, input_features], outputs=output)
    
    # Compile model
    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='binary_crossentropy',
        metrics=['accuracy', tf.keras.metrics.AUC(name='auc')]
    )
    
    return model

# Build the model
model = build_tactical_ai_model(
    sequence_length=X_pos.shape[1],
    pos_features=X_pos.shape[2],
    add_features=X_feat.shape[2]
)

print("Tactical AI Model Architecture:")
model.summary()

## 8. Train the Model

In [ ]:
# Split data into training and validation
from sklearn.model_selection import train_test_split

indices = np.arange(len(y))
train_idx, val_idx = train_test_split(indices, test_size=0.2, random_state=42, stratify=y)

X_pos_train, X_pos_val = X_pos[train_idx], X_pos[val_idx]
X_feat_train, X_feat_val = X_feat[train_idx], X_feat[val_idx]
y_train, y_val = y[train_idx], y[val_idx]

print(f"Training set: {len(y_train)} samples")
print(f"Validation set: {len(y_val)} samples")

In [ ]:
# Train the model
history = model.fit(
    [X_pos_train, X_feat_train],
    y_train,
    validation_data=([X_pos_val, X_feat_val], y_val),
    epochs=20,
    batch_size=32,
    verbose=1
)

## 9. Visualize Training Results

In [ ]:
# Plot training history
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Loss
axes[0].plot(history.history['loss'], label='Training Loss', linewidth=2)
axes[0].plot(history.history['val_loss'], label='Validation Loss', linewidth=2)
axes[0].set_xlabel('Epoch', fontsize=12)
axes[0].set_ylabel('Loss', fontsize=12)
axes[0].set_title('Model Loss', fontsize=14, fontweight='bold')
axes[0].legend(fontsize=10)
axes[0].grid(True, alpha=0.3)

# Accuracy
axes[1].plot(history.history['accuracy'], label='Training Accuracy', linewidth=2)
axes[1].plot(history.history['val_accuracy'], label='Validation Accuracy', linewidth=2)
axes[1].set_xlabel('Epoch', fontsize=12)
axes[1].set_ylabel('Accuracy', fontsize=12)
axes[1].set_title('Model Accuracy', fontsize=14, fontweight='bold')
axes[1].legend(fontsize=10)
axes[1].grid(True, alpha=0.3)

# AUC
axes[2].plot(history.history['auc'], label='Training AUC', linewidth=2)
axes[2].plot(history.history['val_auc'], label='Validation AUC', linewidth=2)
axes[2].set_xlabel('Epoch', fontsize=12)
axes[2].set_ylabel('AUC', fontsize=12)
axes[2].set_title('Model AUC', fontsize=14, fontweight='bold')
axes[2].legend(fontsize=10)
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 10. Functions for Match Simulation

In [ ]:
def simulate_random_match(league: League) -> Dict:
    """Simulate a match between two random teams."""
    
    team1, team2 = random.sample(league.teams, 2)
    goals1, goals2, tracking = league.simulate_match(team1, team2, generate_tracking=True)
    
    result = {
        'team1': team1.name,
        'team2': team2.name,
        'goals1': goals1,
        'goals2': goals2,
        'tracking': tracking
    }
    
    print(f"\n{'='*50}")
    print(f"RANDOM MATCH RESULT")
    print(f"{'='*50}")
    print(f"{team1.name} (Rating: {team1.overall_rating}) vs {team2.name} (Rating: {team2.overall_rating})")
    print(f"Final Score: {goals1} - {goals2}")
    print(f"Tracking sequences generated: {len(tracking)}")
    
    return result

def simulate_specific_matchup(league: League, team1_name: str, team2_name: str) -> Dict:
    """Simulate a match between two specific teams."""
    
    team1 = next((t for t in league.teams if t.name == team1_name), None)
    team2 = next((t for t in league.teams if t.name == team2_name), None)
    
    if not team1 or not team2:
        print(f"Error: One or both teams not found in league.")
        return None
    
    goals1, goals2, tracking = league.simulate_match(team1, team2, generate_tracking=True)
    
    result = {
        'team1': team1.name,
        'team2': team2.name,
        'goals1': goals1,
        'goals2': goals2,
        'tracking': tracking
    }
    
    print(f"\n{'='*50}")
    print(f"SPECIFIC MATCHUP RESULT")
    print(f"{'='*50}")
    print(f"{team1.name} (Rating: {team1.overall_rating}) vs {team2.name} (Rating: {team2.overall_rating})")
    print(f"Final Score: {goals1} - {goals2}")
    print(f"Tracking sequences generated: {len(tracking)}")
    
    return result

In [ ]:
# Test random match simulation
random_match = simulate_random_match(premier_league)

In [ ]:
# Test specific matchup simulation
specific_match = simulate_specific_matchup(premier_league, 'Arsenal', 'Manchester United')

## 11. Visualize Ideal Attack Pattern

In [ ]:
def visualize_ideal_attack_pattern(model, tracking_data: List[Dict], top_n: int = 5):
    """Visualize the top N attack patterns with highest goal probability."""
    
    # Prepare all tracking data for prediction
    X_pos_all, X_feat_all, y_all = prepare_training_data(tracking_data)
    
    # Predict goal probabilities
    predictions = model.predict([X_pos_all, X_feat_all], verbose=0).flatten()
    
    # Get top N sequences by predicted probability
    top_indices = np.argsort(predictions)[-top_n:][::-1]
    
    print(f"\n{'='*60}")
    print(f"TOP {top_n} ATTACK PATTERNS (by predicted goal probability)")
    print(f"{'='*60}\n")
    
    # Create figure with subplots
    fig, axes = plt.subplots(1, min(top_n, 3), figsize=(18, 6))
    if top_n == 1:
        axes = [axes]
    
    for plot_idx, seq_idx in enumerate(top_indices[:3]):
        sequence = tracking_data[seq_idx]
        positions = sequence['positions']
        prob = predictions[seq_idx]
        actual = sequence['is_goal']
        
        print(f"Pattern {plot_idx + 1}:")
        print(f"  Predicted Goal Probability: {prob:.3f}")
        print(f"  Actual Outcome: {'GOAL' if actual else 'No Goal'}")
        print(f"  Team Ratings: {sequence['team1_rating']} vs {sequence['team2_rating']}")
        print()
        
        # Plot on football pitch
        ax = axes[plot_idx]
        
        # Draw pitch
        ax.set_xlim(0, 105)
        ax.set_ylim(0, 68)
        ax.set_aspect('equal')
        
        # Pitch markings
        ax.plot([0, 0], [0, 68], 'k-', linewidth=2)
        ax.plot([105, 105], [0, 68], 'k-', linewidth=2)
        ax.plot([0, 105], [0, 0], 'k-', linewidth=2)
        ax.plot([0, 105], [68, 68], 'k-', linewidth=2)
        ax.plot([52.5, 52.5], [0, 68], 'k--', linewidth=1, alpha=0.5)
        
        # Goal area
        ax.add_patch(plt.Rectangle((105-16.5, 34-20.15), 16.5, 40.3, 
                                   fill=False, edgecolor='red', linewidth=2))
        
        # Plot player trajectories
        colors = plt.cm.rainbow(np.linspace(0, 1, positions.shape[1]))
        
        for player_idx in range(positions.shape[1]):
            trajectory = positions[:, player_idx, :]
            ax.plot(trajectory[:, 0], trajectory[:, 1], 
                   'o-', color=colors[player_idx], alpha=0.6, linewidth=1.5, markersize=3)
            
            # Mark starting position
            ax.plot(trajectory[0, 0], trajectory[0, 1], 
                   'o', color=colors[player_idx], markersize=8, markeredgecolor='black', markeredgewidth=1)
            
            # Mark ending position
            ax.plot(trajectory[-1, 0], trajectory[-1, 1], 
                   's', color=colors[player_idx], markersize=8, markeredgecolor='black', markeredgewidth=1)
        
        ax.set_xlabel('Length (m)', fontsize=10)
        ax.set_ylabel('Width (m)', fontsize=10)
        ax.set_title(f'Pattern {plot_idx + 1}\nGoal Prob: {prob:.3f} | Actual: {"GOAL" if actual else "No Goal"}', 
                    fontsize=11, fontweight='bold')
        ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    return top_indices, predictions[top_indices]

In [ ]:
# Visualize ideal attack patterns
top_patterns, top_probs = visualize_ideal_attack_pattern(model, premier_league.tracking_data, top_n=5)

## 12. Additional Analysis and Visualizations

In [ ]:
# Analyze model predictions on validation set
from sklearn.metrics import classification_report, confusion_matrix

y_pred_proba = model.predict([X_pos_val, X_feat_val], verbose=0).flatten()
y_pred = (y_pred_proba > 0.5).astype(int)

print("\n=== MODEL EVALUATION ON VALIDATION SET ===")
print("\nClassification Report:")
print(classification_report(y_val, y_pred, target_names=['No Goal', 'Goal']))

print("\nConfusion Matrix:")
cm = confusion_matrix(y_val, y_pred)
print(cm)

# Plot confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['No Goal', 'Goal'],
            yticklabels=['No Goal', 'Goal'])
plt.ylabel('True Label', fontsize=12)
plt.xlabel('Predicted Label', fontsize=12)
plt.title('Confusion Matrix - Goal Prediction', fontsize=14, fontweight='bold')
plt.show()

In [ ]:
# Visualize league statistics
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# 1. Goals scored by team
standings = premier_league.get_standings()
axes[0, 0].barh(standings['Team'], standings['GF'], color='green', alpha=0.7)
axes[0, 0].set_xlabel('Goals For', fontsize=12)
axes[0, 0].set_title('Goals Scored by Team', fontsize=14, fontweight='bold')
axes[0, 0].grid(True, alpha=0.3)

# 2. Points distribution
axes[0, 1].barh(standings['Team'], standings['Points'], color='blue', alpha=0.7)
axes[0, 1].set_xlabel('Points', fontsize=12)
axes[0, 1].set_title('League Points', fontsize=14, fontweight='bold')
axes[0, 1].grid(True, alpha=0.3)

# 3. Win-Draw-Loss distribution
x = np.arange(len(standings))
width = 0.25
axes[1, 0].bar(x - width, standings['Wins'], width, label='Wins', color='green', alpha=0.8)
axes[1, 0].bar(x, standings['Draws'], width, label='Draws', color='yellow', alpha=0.8)
axes[1, 0].bar(x + width, standings['Losses'], width, label='Losses', color='red', alpha=0.8)
axes[1, 0].set_ylabel('Number of Matches', fontsize=12)
axes[1, 0].set_title('Match Results Distribution', fontsize=14, fontweight='bold')
axes[1, 0].set_xticks(x)
axes[1, 0].set_xticklabels(standings['Team'], rotation=45, ha='right')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# 4. Rating vs Points scatter
axes[1, 1].scatter(standings['Rating'], standings['Points'], s=200, alpha=0.6, c=standings['GF'], cmap='viridis')
for idx, row in standings.iterrows():
    axes[1, 1].annotate(row['Team'], (row['Rating'], row['Points']), 
                       fontsize=9, ha='center', va='bottom')
axes[1, 1].set_xlabel('Team Rating', fontsize=12)
axes[1, 1].set_ylabel('Points', fontsize=12)
axes[1, 1].set_title('Team Rating vs Points', fontsize=14, fontweight='bold')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 13. Summary and Conclusions

In [ ]:
print("="*70)
print("FOOTBALL LEAGUE SIMULATION AND TACTICAL AI - SUMMARY")
print("="*70)
print()
print("✓ Created Player, Team, and League classes with random stats")
print("✓ Simulated a complete season with multiple teams")
print(f"✓ Generated {len(premier_league.tracking_data)} tracking sequences")
print("✓ Built BiLSTM + Multi-Head Attention model for goal prediction")
print("✓ Trained model on match tracking data")
print("✓ Implemented functions for random and specific match simulations")
print("✓ Visualized ideal attack patterns based on model predictions")
print()
print("Key Results:")
print(f"  - Total matches played: {len(premier_league.match_results)}")
print(f"  - Model validation accuracy: {history.history['val_accuracy'][-1]:.3f}")
print(f"  - Model validation AUC: {history.history['val_auc'][-1]:.3f}")
print()
print("Champion Team:")
champion = standings.iloc[0]
print(f"  {champion['Team']} - {champion['Points']} points ({champion['Wins']}W-{champion['Draws']}D-{champion['Losses']}L)")
print()
print("="*70)